<a href="https://colab.research.google.com/github/pierrobernes/PV_System_conception/blob/main/Conception_PV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# A CONFIGURER PAR INSTALLATION
#--------------------------------
# Installation details
customer_annual_consumption = 5000 #kwh
roof_inclination = [35,35]
roof_orientation = ["Ouest", "Est"]
surface_repartition = [0.5,0.5]
panel_repartition = [7,7] #Panel repartition on the roof
n_panels_installed = [7,7] #String configuration
dc_line_length = [10,10]
ac_line_length = 5
selected_config_text = "Etant donné l'orientation des versants de toiture en EST-OUEST, une configuration de 7 panneaux sur chaque versant est retenue.  Les panneaux de chaque versant seront raccordés ensemble sur un string respectif ce qui est compatible avec l'onduleur (cfr. section précédente)"

# PV panels configuration
pv_panel_brand = "Hyundai"
pv_max_power = 435
pv_panel_model = f"HiT-H{pv_max_power}MF-FB"
pv_width = 1.134 #m
pv_length = 1.722 #m

#PV STC values
pv_v_oc_stc = 42.18
pv_i_sc_stc = 13.10
pv_v_mpp_stc = 35.38
pv_i_mpp_stc = 12.58
pv_ct = -0.24 #(% V_oc/deg)
pv_module_efficiency = 0.2279

#PV NOCT values
pv_v_oc_noct = 40.26
pv_i_sc_noct = 10.56
pv_v_mpp_noct = 33.34
pv_i_mpp_noct = 10.14

#Inverter features
inverter_brand = "Huawei"
inverter_type = "Sun2000-4KTL-M1 (Haute intensité)"

inverter={
    "max_p_dc": {"value": 6000, "alias": "Puissance PV Max (W)"},
    "max_v_dc_off": {"value": 1100, "alias": "Tension d'entrée maximum (V)"},
    "min_v_dc_run": {"value": 140, "alias": "Plage de tension de fonctionnement - Min (V)"},
    "max_v_dc_run": {"value": 980, "alias": "Plage de tension de fonctionnement - Max (V)"},
    "min_startup_v_dc": {"value": 200, "alias": "Tension minimum de démarrage (V)"},
    "nominal_v_dc": {"value": 600, "alias": "Tension nominale de fonctionnement (V)"},
    "max_i_dc_mpp_mppt": {"value": 13.5, "alias": "Courant maximum par MPPT au MPP(A)"},
    "max_i_dc_sc_mppt": {"value": 19.5, "alias": "Courant maximum par MPPT en court-circuit(A)"},
    "ac_output_type": {"value": "3N400", "alias": "Type de sortie AC "}, #MONO, 3N400, 3X230
    "max_i_ac": {"value": 6.8, "alias": "Courant de sortie maximum (A)"},
    "efficiency": {"value": 0.975, "alias": "Rendement énergétique"},
    "mppt": {"value": 2, "alias": "Nombre de MPPT"},
}

In [27]:
# DON'T TOUCH THIS UNLESS YOU KNOW WHAT YOU DO
#----------------------------------------------

# Tool Configs
config_kwh_kwp = 1000
max_allowed_v_dc = 750
stc_temp = 25
t_min = -10
t_max = 70
output_filename = "pv_report.md"
max_loss = 0.02
max_voltage_drop = 0.01
r_dc_wire_4 = 5.09 #ohms/km
r_dc_wire_6 = 3.39 #ohms/km
r_ac_wire = 0.0169 #ohms*mm²/m
l_panel_wires = 1.2 #m
panel_spacing = 0.02 # m
panel_wire_section = 4 #mm²
WIRE_SECTION_LIST = [1.5, 2.5, 4, 6, 10, 16, 1000000]
plug_efficiency = 0.98
meter_efficiency = 0.98
electric_install_efficiency = 0.98

#Alias
variable_aliases = {
    "pv_v_oc": "V<sub>oc</sub> (V)",
    "pv_i_sc": "I<sub>sc</sub> (A)",
    "pv_v_mpp": "V<sub>mpp</sub> (V)",
    "pv_i_mpp": "I<sub>mpp</sub> (A)"
}

# Dictionaries
orientation_dict = {
    "est": -90,
    "sud-est": -45,
    "sud": 0,
    "sud-ouest": 45,
    "ouest": 90
}

correction_factor = {
    '-90,0': 0.88, '-90,15': 0.87, '-90,25': 0.85, '-90,35': 0.83, '-90,50': 0.77, '-90,70': 0.65, '-90,90': 0.50,
    '-45,0': 0.88, '-45,15': 0.93, '-45,25': 0.95, '-45,35': 0.95, '-45,50': 0.92, '-45,70': 0.81, '-45,90': 0.64,
    '0,0': 0.88, '0,15': 0.96, '0,25': 0.99, '0,35': 1.00, '0,50': 0.98, '0,70': 0.87, '0,90': 0.68,
    '45,0': 0.88, '45,15': 0.93, '45,25': 0.95, '45,35': 0.95, '45,50': 0.92, '45,70': 0.91, '45,90': 0.64,
    '90,0': 0.88, '90,15': 0.87, '90,25': 0.85, '90,35': 0.82, '90,50': 0.76, '90,70': 0.65, '90,90': 0.5,
}

grid_voltages = {
    "3N400": 400,
    "3x230": 230,
    "1N400": 230,
    "2x230": 230
}

In [28]:
from IPython.display import Markdown, display
import math
import re
import os
import sys

_markdown_file = None
_markdown_filename = None

def remove_file_if_exists(filename='pv_report_output.md'):
  """
  Checks if a file exists and deletes it if it does.

  Args:
    filename (str): The full or relative path to the file to be deleted.
                    Defaults to 'pv_report_output.md'.
  """
  print(f"Checking for the existence of file: '{filename}'")

  # os.path.exists() checks if a path (file or directory) exists
  if os.path.exists(filename):
    # Check if it's actually a file and not a directory (additional safety)
    if os.path.isfile(filename):
      try:
        # os.remove() deletes the file
        os.remove(filename)
        print(f"File '{filename}' existed and was successfully deleted.")
      except OSError as e:
        # Handle potential errors during deletion (e.g., permissions)
        print(f"Error while trying to delete file '{filename}': {e}", file=sys.stderr)
      except Exception as e:
        # Handle any other unexpected errors
        print(f"An unexpected error occurred during deletion: {e}", file=sys.stderr)
    else:
      # The path exists but is not a file (it's a directory)
      print(f"Path '{filename}' exists but it is not a file. No file deletion action taken.")
  else:
    # The file does not exist
    print(f"File '{filename}' does not exist. No deletion needed.")

def md_dis_wr(markdown_text, filename="pv_report.md", append=True):
    """
    Affiche du Markdown dans le notebook et l'ajoute à un fichier.

    Args:
        markdown_text (str): La chaîne de texte au format Markdown à afficher et à ajouter.
        filename (str, optional): Le nom du fichier .md. Par défaut "output.md".
        append (bool, optional): Indique si le texte doit être ajouté à la fin du fichier
                                 (True) ou si le fichier doit être écrasé (False). Par défaut True.
    """
    global _markdown_file
    global _markdown_filename

    display(Markdown(markdown_text))

    mode = "a" if append else "w"

    if _markdown_file is None or _markdown_filename != filename:
        if _markdown_file is not None:
            _markdown_file.close()
        try:
            _markdown_file = open(filename, mode, encoding="utf-8")
            _markdown_filename = filename
        except Exception as e:
            print(f"Erreur lors de l'ouverture du fichier '{filename}': {e}")
            _markdown_file = None
            _markdown_filename = None
            return

    if _markdown_file:
        try:
            _markdown_file.write(markdown_text + "\n\n")
        except Exception as e:
            print(f"Erreur lors de l'écriture dans le fichier '{filename}': {e}")

def close_md_file():
    """
    Ferme le fichier Markdown s'il est ouvert.
    """
    global _markdown_file
    if _markdown_file is not None:
        _markdown_file.close()
        _markdown_file = None
        global _markdown_filename
        _markdown_filename = None
    else:
        print("Aucun fichier Markdown n'était ouvert.")

def pv_markdown_table():
    table = "| Caractéristique | Valeur STC | Valeur NOCT |\n"
    table += "|---|---|---| \n"
    for var_name in variable_aliases:
      alias = variable_aliases[var_name]
      table += f"| {alias} | {globals()[f'{var_name}_stc']} | {globals()[f'{var_name}_noct']} |\n"

    return table

def create_roof_table_markdown(roof_inclination, roof_orientation, surface_repartition):

    table = "| Surface | Inclinaison (°)| Orientation | Repartition (%) |\n"
    table += "|---------|---------------|-------------|---------------|\n"

    for i in range(len(roof_inclination)):
        inclination = roof_inclination[i]
        orientation = roof_orientation[i]
        repartition = surface_repartition[i] * 100
        table += f"| {i+1}       | {inclination}          | {orientation}   | {repartition:.1f}        |\n"

    return table

def display_inverter_properties_markdown(inverter_data):
    table = "| Caractéristique                     | Valeur              |\n"
    table += "|-------------------------------------|---------------------|\n"

    for key, details in inverter_data.items():
        alias = details.get("alias", key)
        value = details.get("value", "N/A")
        table += f"| {alias:<35} | {value:<20} |\n"

    return table

def get_correction_factor(orientation: str, inclination: int) -> float:

    orientation_angle = orientation_dict.get(orientation.lower())
    if orientation_angle is None:
        return None

    relevant_factors = {
        int(k.split(',')[1]): v for k, v in correction_factor.items() if int(k.split(',')[0]) == orientation_angle
    }

    if not relevant_factors:
        return None

    inclinations = sorted(relevant_factors.keys())

    if inclination in relevant_factors:
        return relevant_factors[inclination]

    if inclination < inclinations[0]:
        return None
    if inclination > inclinations[-1]:
        return None

    lower_inclination = max(i for i in inclinations if i < inclination)
    upper_inclination = min(i for i in inclinations if i > inclination)

    lower_factor = relevant_factors[lower_inclination]
    upper_factor = relevant_factors[upper_inclination]

    f_c = lower_factor + (
        (upper_factor - lower_factor) * (inclination - lower_inclination) / (upper_inclination - lower_inclination)
    )

    formula = rf"$$F_c = {lower_factor} + ({inclination}-{lower_inclination}) * \frac{{ {upper_factor}-{lower_factor} }}{{{upper_inclination}-{lower_inclination}}} = {f_c}$$"
    md_dis_wr("Détermination du facteur de correction $F_c$ en fonction de l'orientation et l'inclinaison :")
    md_dis_wr(formula)

    return f_c

def get_production_needs(customer_annual_consumption: int, roof_inclination: list, roof_orientation: list, surface_repartition: list) -> list:

  results = []
  fcs = []
  for i in range(len(roof_inclination)):
    inclination = roof_inclination[i]
    orientation = roof_orientation[i]
    repartition = surface_repartition[i]

    md_dis_wr(f"#### Toiture {i+1}")
    md_dis_wr(f"La surface de toiture a les caractéristiques suivantes :  <ul><li> Inclinaison {inclination}°, <li> Orientation {orientation}, <li> Répartition : {repartition*100}% de la production</ul>")

    f_c = get_correction_factor(orientation, inclination)
    fcs.append(f_c)

    production_needs = round((customer_annual_consumption * repartition) / (config_kwh_kwp * f_c), 2)

    formula = rf"$$ Puissance\ crete = \frac{{ {customer_annual_consumption}* {repartition} }} {{ {config_kwh_kwp} * {f_c}}} = {production_needs} (Kwc)$$"
    md_dis_wr("Estimation de la puissance crete nécessaire")
    md_dis_wr(formula)

    results.append(production_needs)

  return results,fcs

def get_panel_number(peak_power, panel_max_power) -> int:
  pv_number = peak_power / (panel_max_power/1000)
  return math.ceil(pv_number)

def get_voc_for_temp(v_oc: float, c_t: float, temp: float) -> float:
  v_oc_temp = round(v_oc - (v_oc*(c_t/100)*(stc_temp-temp)),2)
  md_dis_wr(rf"$$V_{{oc({temp})}} = {v_oc} - ({v_oc}*{c_t}*({stc_temp}-{temp})) = {v_oc_temp}\ (V)$$")
  return v_oc_temp

def get_vmpp_for_temp(v_mpp: float, v_oc: float, c_t: float, temp: float) -> float:
  v_mpp_temp = round(v_mpp - (v_oc*(c_t/100)*(stc_temp-temp)),2)
  md_dis_wr(rf"$$V_{{mpp({temp})}} = {v_mpp} - ({v_oc}*{c_t}*({stc_temp}-{temp})) = {v_mpp_temp}\ (V)$$")
  return v_mpp_temp

def render_table_panels(n_min: int, n_max: int) -> str:
  header =    "| Description                          | Valeur |"
  separator = "|--------------------------------------|--------|"
  row_min =   f"| Min. panneaux / string  | {n_min:<6} |"
  row_max =   f"| Max. panneaux / string  | {n_max:<6} |"
  markdown_table_string = f"{header}\n{separator}\n{row_min}\n{row_max}"
  return markdown_table_string

def render_current_panels() -> str:
  header = "| Courant panneaux                        | Norme | Compatibilité | Résultat |"
  separator = "|:-------------------------------------|--------|----|----|"
  row_isc_stc =   f"| $I_{{sc}}$  | STC | {pv_i_sc_stc} A <= {inverter['max_i_dc_sc_mppt']['value']} A ?| {'OK' if pv_i_sc_stc<inverter['max_i_dc_sc_mppt']['value'] else 'KO'}|"
  row_impp_stc =   f"| $I_{{mpp}}$ | STC| {pv_i_mpp_stc} A <= {inverter['max_i_dc_mpp_mppt']['value']} A ?| {'OK' if pv_i_mpp_stc<inverter['max_i_dc_mpp_mppt']['value'] else 'KO'}|"
  row_isc_noct =   f"| $$I_{{sc}}$$  | NOCT | {pv_i_sc_noct} A <= {inverter['max_i_dc_sc_mppt']['value']} A ? | {'OK' if pv_i_sc_noct<inverter['max_i_dc_sc_mppt']['value'] else 'KO'}|"
  row_impp_noct =   f"| $$I_{{mpp}}$$ | NOCT| {pv_i_mpp_noct} A <= {inverter['max_i_dc_mpp_mppt']['value']} A ?| {'OK' if pv_i_mpp_noct<inverter['max_i_dc_mpp_mppt']['value'] else 'KO'}|"
  markdown_table_string = f"{header}\n{separator}\n{row_isc_stc}\n{row_impp_stc}\n{row_isc_noct}\n{row_impp_noct}"
  return markdown_table_string

def compute_and_render_string_length(dc_line_length : list, n_panels_string : list, s : int) :
  table =    "|String| Cable                       | Méthode | Longueur| Section|\n"
  table +=   "|------|:----------------------------|:--------|---------:|-------:|\n"
  l_tot_s = 0
  l_tot_wire_panels = 0
  for i in range(len(dc_line_length)) :
    table += f"|[{i+1}] | Longueur onduleur - string| $$ 2*{dc_line_length[i]} m $$ | {round(dc_line_length[i]*2,1)} m | {s} mm² |\n"
    l_tot_s += round(dc_line_length[i]*2,1)
    table +=f"|[{i+1}] | Cables de panneaux| $$ {n_panels_string[i]} * {l_panel_wires} m * 2 $$ | {round(n_panels_string[i]*l_panel_wires*2,1)} m | {panel_wire_section} mm²|\n"
    l_tot_wire_panels += round(n_panels_string[i]*l_panel_wires*2,1)
    table +=f"|[{i+1}] | Brin retour string| $$ ({n_panels_string[i]} * {pv_width + panel_spacing} m)- {panel_spacing} m $$ | {round((n_panels_string[i]*(pv_width + panel_spacing))-panel_spacing,1)} m | {s} mm²|\n"
    l_tot_s += round((n_panels_string[i]*(pv_width + panel_spacing))-panel_spacing,1)

  return table, l_tot_s, l_tot_wire_panels

def find_first_greater_in_list( n,number_list = WIRE_SECTION_LIST) :

    for item in number_list:
        if item > n:
            return item
    return None

def get_ac_max_section(max_loss, dc_loss, i_ac_max, l_ac, grid):
  max_ac_loss = round(max_loss - dc_loss,2)
  text = "\n\n" + f"$$Pertes_{{AC\ Max}} = Perte_{{Système\ Max}} - Perte_{{DC\ MAX}} = {max_loss} W - {dc_loss} W = {max_ac_loss} W$$"

  text += "\n\n" + rf"La résistance maximale pour cette perte est obtenue par la formule suivante:"
  r_max = 0

  if grid in("3N400","3x230"):
    r_max =max_ac_loss/(3*(i_ac_max ** 2))
    text += "\n\n" + rf"$$Pertes_{{AC\ Max}} = 3*R_{{Max\ cable\ AC}}*I_{{AC\ Max}}^2 \Rightarrow R_{{Max\ cable\ AC}} = \frac{{Pertes_{{AC\ Max}}}}{{3*I_{{AC\ Max}}^2}} = \frac{{{max_ac_loss}}}{{3*{inverter['max_i_ac']['value']}^2}} = {round(r_max,3)} \Omega$$"
  else:
    r_max = max_ac_loss/(2*(i_ac_max ** 2))
    text += "\n\n" + rf"$$Pertes_{{AC\ Max}} = 2*R_{{Max\ cable\ AC}}*I_{{AC\ Max}}^2 \Rightarrow R_{{Max\ cable\ AC}} = \frac{{Pertes_{{AC\ Max}}}}{{2*I_{{AC\ Max}}^2}} = \frac{{{max_ac_loss}}}{{2*{inverter['max_i_ac']['value']}^2}} = {round(r_max,3)} \Omega$$"

  text += "\n\n" + rf"La section minimale du cable AC est alors obtenue par la loi de Pouillet (avec une résistivité pour le cable de cuivre établie à {r_ac_wire} $ \frac{{\Omega*mm^2}}{{m}}$ : "

  s_min_ac = r_ac_wire*l_ac/round(r_max,3)
  text += "\n\n" + rf"$$S_{{min}} = \frac{{{r_ac_wire} * l_{{cable\ ac}}}}{{ R_{{Max\ cable\ AC}}}} = \frac{{{r_ac_wire} * {l_ac}}}{{{round(r_max,3)}}} = {round(s_min_ac,2)} mm^2 $$"

  wire_section = find_first_greater_in_list(s_min_ac)

  text +="\n\n" + rf"**La section minimum du cable AC a utiliser pour limiter les pertes de ligne est donc de {wire_section}mm²**"

  return text, wire_section

def get_ac_loss(i_ac_max, l_ac, wire_section, grid):
  if grid in("3N400","3x230"):
    ac_loss =3*(r_ac_wire*l_ac/wire_section)*(i_ac_max ** 2)
  else:
    ac_loss =2*(r_ac_wire*l_ac/wire_section)*(i_ac_max ** 2)
  return round(ac_loss,1)

def get_ac_section_for_voltage_drop(i_ac_max, l_ac, grid):
  voltage_drop_value=round(max_voltage_drop*grid_voltages[grid],1)
  s_min = 0
  text = rf"La chute de tension maximum se calcule comme suit (en négligeant l'impédance de ligne (section < 50mm²) selon le réseau ({grid}):)"
  if grid in("3N400","3x230"):
    text += "\n\n" + rf"$$Reseau\ Tri\ {grid} \Rightarrow U_{{max}} = \sqrt(3)*R*I_{{AC\ Max}} = {max_voltage_drop*100}\% * {grid_voltages[grid]} V = {voltage_drop_value} V$$"
    r_max = round(voltage_drop_value/(math.sqrt(3)*i_ac_max),4)
    formula_r_max = rf"$$R_{{max}}=\frac{{ U_{{max}} }}{{I_{{AC\ Max}}}} = \frac{{{voltage_drop_value} V}}{{\sqrt(3)*{i_ac_max} A}} = {r_max} \Omega$$"
    s_min = round((r_ac_wire*3*l_ac)/r_max,2)
    formula_s_min = rf"$$S_{{min}}=\frac{{\rho * l_{{3\ fils}}}}{{R_{{max}}}}=\frac{{{r_ac_wire}*(3*{l_ac}) m}}{{{r_max} \Omega}} = {s_min} mm²$$"
  else:
    text += "\n\n" + rf"$$Reseau\ Tri\ {grid} \Rightarrow U_{{max}} = R*I_{{AC\ Max}} = {max_voltage_drop*100}\% * {grid_voltages[grid]} V = {voltage_drop_value} V$$"
    r_max = round(voltage_drop_value/(i_ac_max),4)
    formula_r_max = rf"$$R_{{max}}=\frac{{ U_{{max}} }}{{I_{{AC\ Max}}}} = \frac{{{voltage_drop_value} V}}{{{i_ac_max} A}} = {r_max} \Omega$$"
    s_min = round((r_ac_wire*2*l_ac)/r_max,2)
    formula_s_min = rf"$$S_{{min}}=\frac{{\rho * l_{{2\ fils}}}}{{R_{{max}}}}=\frac{{{r_ac_wire}*(2*{l_ac}) m}}{{{r_max} \Omega}} = {s_min} mm²$$"

  text += "\n\n" + rf"On recherche alors la résistance la résistance maximum du cable pour respecter cette chute de tension maximum :"
  text += "\n\n" + formula_r_max
  text += "\n\n" + rf"Sur base de cela il ne reste qu'à calculer la section de fil avec la loi de Pouillet : "
  text += "\n\n" + formula_s_min
  wire_section = find_first_greater_in_list(s_min)
  text +="\n\n" + rf"**La section minimum du cable AC a utiliser pour limiter les pertes de ligne est donc de {wire_section}mm²**"

  return text,wire_section

def render_dc_wire_comptutation(table_wire_length_6,l_tot_6_6,l_tot_6_4,pv_i_mpp_stc):
    md_dis_wr(table_wire_length_6)
    md_dis_wr(rf"La longueur totale de cable : ")
    md_dis_wr(f"<ul>")
    md_dis_wr(rf"<li>en 6mm² : {l_tot_6_6}m")
    md_dis_wr(rf"<li>en 4mm² : {l_tot_6_4}")
    md_dis_wr(f"</ul>")
    dc_loss_6 = round((r_dc_wire_6*1e-3*l_tot_6_6+r_dc_wire_4*1e-3*l_tot_6_4)*pv_i_mpp_stc*pv_i_mpp_stc,1)
    md_dis_wr(rf"$$Pertes_{{DC}} = {round(r_dc_wire_6*1e-3,5)} Ohms/m * {l_tot_6_6} m * {pv_i_mpp_stc} A^2 + {round(r_dc_wire_4*1e-3,5)} Ohms/m * {l_tot_6_4} m * {pv_i_mpp_stc} A^2= {dc_loss_6} W$$")
    dc_loss = dc_loss_6
    md_dis_wr(rf"**La section de cable retenue pour les cables DC est : 6mm²**")
    return dc_loss

def get_annual_production(panels, fcs):
  prod =[]
  for i in range(len(fcs)):
    prod.append(round(panels[i] * (pv_max_power/1000) * fcs[i]*config_kwh_kwp,2))
  return prod


In [29]:
import re
import os
import sys

def transform_dollar_delimiters(input_filename='pv_report.md', output_filename='pv_report_output.md'):
  """
  Lit un fichier, remplace les délimiteurs '$...$' par '$$...$$' SEULEMENT s'ils ne
  sont pas déjà doubles, et écrit le résultat dans un autre fichier.

  Args:
    input_filename (str): Le chemin vers le fichier d'entrée.
                            Par défaut 'pv_report.md'.
    output_filename (str): Le chemin vers le fichier de sortie.
                             Par défaut 'pv_report_output.md'.
  """

  # Fonction de remplacement pour re.sub
  # Elle reçoit un objet 'match' et décide quoi retourner.
  def replacement_logic(match):
    # match.group(1) correspond à la capture de (\$\$(.*?)\$\$)
    # Si group(1) existe, c'est qu'on a trouvé $$...$$, il ne faut rien changer.
    if match.group(1):
      return match.group(0) # Retourne la correspondance entière ($$...$$) telle quelle

    # Sinon, c'est que match.group(3) a fonctionné (\$(.*?)\$)
    # match.group(4) contient le contenu *entre* les dollars simples.
    else:
      content = match.group(4)
      # On retourne le contenu entouré de doubles dollars
      return f"$${content}$$"

  # Regex combinée :
  # 1. (\$\$(.*?)\$\$) : Capture les blocs $$...$$ dans le groupe 1 (et leur contenu dans le groupe 2)
  # 2. | : OU
  # 3. (\$(.*?)\$) : Capture les blocs $...$ dans le groupe 3 (et leur contenu dans le groupe 4)
  # L'ordre est important : on essaie de matcher d'abord $$...$$ pour pouvoir l'ignorer.
  regex_combined = r"(\$\$(.*?)\$\$)|(\$(.*?)\$)"

  # --- Début du traitement --- (Messages en français)
  print(f"--- Début du traitement ---")
  print(f"Fichier d'entrée : {input_filename}")
  print(f"Fichier de sortie : {output_filename}")

  try:
    # Lire le contenu du fichier d'entrée
    print(f"Lecture du fichier '{input_filename}'...")
    with open(input_filename, 'r', encoding='utf-8') as infile:
      original_content = infile.read()
    print("Lecture terminée.")

    # Exécuter le remplacement en utilisant la fonction de logique
    print("Exécution des remplacements conditionnels...")
    modified_content = re.sub(regex_combined, replacement_logic, original_content)

    # Vérifier si des modifications ont été faites
    if original_content == modified_content:
        print("Aucune modification pertinente détectée.")
    else:
        print("Remplacements effectués.")

    # Écrire le contenu modifié dans le fichier de sortie
    print(f"Écriture dans le fichier '{output_filename}'...")
    with open(output_filename, 'w', encoding='utf-8') as outfile:
      outfile.write(modified_content)
    print("Écriture terminée.")

  except FileNotFoundError:
    print(f"Erreur : Le fichier d'entrée '{input_filename}' n'a pas été trouvé.")
    print("Traitement annulé.")
  except IOError as e:
    print(f"Erreur d'entrée/sortie lors de l'accès aux fichiers : {e}")
    print("Traitement annulé.")
  except Exception as e:
    print(f"Une erreur inattendue est survenue : {e}")
    print("Traitement annulé.")

  print(f"--- Fin du traitement ---")




In [30]:
remove_file_if_exists(filename='pv_report_output.md')
transform_dollar_delimiters(input_filename='pv_report.md', output_filename='pv_report_output.md')

Checking for the existence of file: 'pv_report_output.md'
File 'pv_report_output.md' does not exist. No deletion needed.
--- Début du traitement ---
Fichier d'entrée : pv_report.md
Fichier de sortie : pv_report_output.md
Lecture du fichier 'pv_report.md'...
Lecture terminée.
Exécution des remplacements conditionnels...
Remplacements effectués.
Écriture dans le fichier 'pv_report_output.md'...
Écriture terminée.
--- Fin du traitement ---


In [31]:
# @title Default title text
import math

remove_file_if_exists(filename='pv_report.md')
remove_file_if_exists(filename='pv_report_output.md')

md_dis_wr("# 1. Description de l'installation")

md_dis_wr(f"Le client projette une consommation de {customer_annual_consumption} Kwh.")


md_dis_wr(
    f"Les surfaces de toiture exploitables sont les suivantes : \
    "
)

md_dis_wr(create_roof_table_markdown(roof_inclination, roof_orientation, surface_repartition))

md_dis_wr("# 2. Estimation de la puissance crete nécessaire")
md_dis_wr(f"La puissance crete nécessaire est estimée sur base de :")
md_dis_wr(f"<ul>")
md_dis_wr(f"<li>la consommation annuelle")
md_dis_wr(f"<li>une production forfaitaire de 1000Kwh par Kwc de panneaux")
md_dis_wr(f"<li>un facteur correctif $F_c$ fonction de l'inclinaison et de l'orientation")
md_dis_wr(f"Afin de tenir compte de toitures d'orientation différentes, un facteur de répartition $F_r$ pondère le calcul par orientation")
md_dis_wr(f"</ul>")

md_dis_wr(rf"$$ Puissance\ crete\ par\ orientation= \frac{{Consommation\ annuelle * F_r}}{{1000 * F_c}} $$")

md_dis_wr("## 2.1 Calcul")

production_needs,fcs = get_production_needs(customer_annual_consumption, roof_inclination, roof_orientation, surface_repartition)
total_production_needs = round(sum(production_needs),2)

md_dis_wr(f"**La puissance crete totale requise est de {total_production_needs} Kwc**")

md_dis_wr("## 2.2 Détermination du nombre de panneaux nécessaires")

md_dis_wr(f"Les panneaux utilisés sont de marque **{pv_panel_brand}** et série **{pv_panel_model}**.")
md_dis_wr(f"Leur puissance crete maximale est de {pv_max_power} W.  Le nombre de panneaux peut donc être calculé comme suit : ")
panel_number = get_panel_number(total_production_needs, pv_max_power)
md_dis_wr(rf"$$ Puissance\ crete = \frac{{{total_production_needs * 1000}\ (W)}}{{{pv_max_power}\ (W)}} = {panel_number}\ (arrondi\ superieur)$$")

pv_total_peak_power = panel_number * pv_max_power
md_dis_wr(f"**Le nombre de panneaux nécessaires est de {panel_number}**; ce qui permettra de délivrer une puissance crete cumulée de **{pv_total_peak_power /1000} Kwc.**")

md_dis_wr("# 3. Analyse de compatibilité avec le matériel utilisé")
md_dis_wr("## 3.1 Description du matériel")
md_dis_wr("### 3.1.1 Panneaux")

md_dis_wr(f"Les panneaux utilisés sont de marque **{pv_panel_brand}** et série **{pv_panel_model}**.\
 Leurs caractéristiques sont reprises dans la table ci-dessous : ")
md_dis_wr(pv_markdown_table())
md_dis_wr(f"Coefficient de température V<sub>oc</sub>: **{pv_ct}** %/°C")

md_dis_wr("### 3.1.2 Onduleur")

md_dis_wr(f"L'onduleur utilisé est de marque **{inverter_brand}** et de série **{inverter_type}**.\
 Ses caractéristiques sont reprises dans la table ci-dessous : ")
inverter_table = display_inverter_properties_markdown(inverter)
md_dis_wr(inverter_table)

md_dis_wr("## 3.2 Calculs")
md_dis_wr("### 3.2.1 Compatibilité en tension")
md_dis_wr(f"Les tensions dans les conditions de température au niveau de la cellule les plus défavorables pour l'onduleur sont considérées :")
md_dis_wr(f"<ul>")
md_dis_wr(f"<li> par temps froid (cellule PV à {t_min}°C) - la tension augmente sur les cellules; la somme de ces tensions sur le string pourrait dépasser les tolérances de l'onduleur")
md_dis_wr(f"<li> par temps chaud (cellule PV à {t_max}°C) - la tension diminue sur les cellules et la limite basse de démarrage de l'onduleur pourrait ne pas être atteinte")
md_dis_wr(f"</ul>")

md_dis_wr(f"En effet, les tensions renseignées par la norme STC sont faites à température de cellule de {stc_temp}°C.")
md_dis_wr(r"Le coefficient de variation de la tension à vide $V_{{oc}}$ appelé ci-après $C_t$ est utilisé dans les formules ci-dessous pour déterminer les tensions à vide au niveau de température de cellule souhaité: ")

md_dis_wr(rf"$$V_{{oc(temp)}} = V_{{oc}} - (V_{{oc}}*C_t*({stc_temp}-temp))$$")

md_dis_wr(rf"Par température de cellule de {t_min}°C, le $V_{{oc}}$ devient : ")
v_oc_t_min = get_voc_for_temp(pv_v_oc_stc, pv_ct, t_min)

md_dis_wr(rf"A cette température, le $V_{{mpp}}$ devient :  ")
v_mpp_t_min = get_vmpp_for_temp(pv_v_mpp_stc, pv_v_oc_stc, pv_ct, t_min)

md_dis_wr(rf"Tandis que par température de cellule de {t_max}°C, le $V_{{oc}}$ devient : ")
v_oc_t_max = get_voc_for_temp(pv_v_oc_stc, pv_ct, t_max)

md_dis_wr(rf"Grâce à ces valeurs, on peut déterminer le nombre maximum de panneau par string :")

max_p_per_string = round(inverter['max_v_dc_off']['value'] / v_oc_t_min, 2)
md_dis_wr(rf"$$\frac{{DC_{{max\ onduleur\ a\ vide}}}}{{V_{{oc(-10)}}}} = \frac{{{inverter['max_v_dc_off']['value']}\ (V)}} {{{v_oc_t_min}\ (V)}} = {max_p_per_string} \Rightarrow Max.\ {math.floor(max_p_per_string)}\ Panneaux\ par\ string$$")
if inverter['max_v_dc_off']['value'] >= max_allowed_v_dc:
  md_dis_wr(rf"**Attention** cependant car la tension max. admissible à vide de l'onduleur ({inverter['max_v_dc_off']['value']} V) est supérieure à la tension admissible sur toiture résidentielle ({max_allowed_v_dc} V).")
  md_dis_wr(rf"Le nombre de panneaux maximum à vide doit donc être revu sur base de cette tension : ")
  max_p_per_string = round(max_allowed_v_dc / v_oc_t_min, 2)
  md_dis_wr(rf"$$\frac{{Tension\ Max\ sur\ toiture}}{{V_{{oc(-10)}}}} = \frac{{{max_allowed_v_dc}\ (V)}} {{{v_oc_t_min}\ (V)}} = {max_p_per_string} \Rightarrow Max.\ {math.floor(max_p_per_string)}\ Panneaux\ par\ string$$")

md_dis_wr(rf"Pour garantir le démarrage de l'onduleur par temps chaud, le nombre minimum de panneaux est de :")

min_p_per_string = round(inverter['min_v_dc_run']['value'] / v_oc_t_max, 2)
md_dis_wr(rf"$$\frac{{DC_{{min\ démarrage\ onduleur}}}}{{V_{{oc(70)}}}} = \frac{{{inverter['min_v_dc_run']['value']}\ (V)}} {{{v_oc_t_max}\ (V)}} = {min_p_per_string} \Rightarrow Min.\ {math.ceil(min_p_per_string)}\ Panneaux\ par\ string$$")

md_dis_wr(rf"En fonctionnement par temps froid, le nombre de panneaux par string ne pourra pas dépasser :")

if inverter['max_v_dc_off']['value'] >= max_allowed_v_dc:
  max_p_per_string_run = round(max_allowed_v_dc / v_mpp_t_min, 2)
  md_dis_wr(rf"$$\frac{{Tension\ Max\ sur\ toiture}}{{V_{{mpp(-10)}}}} = \frac{{{max_allowed_v_dc}\ (V)}} {{{v_mpp_t_min}\ (V)}} = {max_p_per_string_run} \Rightarrow Max.\ {math.floor(max_p_per_string_run)}\ Panneaux\ par\ string$$")
else:
  max_p_per_string_run = round(inverter['max_v_dc_run']['value'] / v_mpp_t_min, 2)
  md_dis_wr(rf"$$\frac{{DC_{{max\ onduleur\ run}}}}{{V_{{mpp(-10)}}}} = \frac{{{inverter['max_v_dc_run']['value']}\ (V)}} {{{v_mpp_t_min}\ (V)}} = {max_p_per_string_run} \Rightarrow Max.\ {math.floor(max_p_per_string_run)}\ Panneaux\ par\ string$$")

md_dis_wr("#### Conclusions")
n_p_max = min([math.floor(max_p_per_string), math.floor(max_p_per_string_run)])
n_p_min = math.ceil(min_p_per_string)
dc_power_max = n_p_max * pv_max_power
md_dis_wr(render_table_panels(n_p_min, n_p_max))

if (dc_power_max > inverter['max_p_dc']['value']):
  md_dis_wr(rf"La puissance DC maximum recommandée (sans optimiseur) est donnée par le fabricant à {inverter['max_p_dc']['value']} W.  Mettre {n_p_max} panneaux produirait {dc_power_max} W.  Idéalement il faudrait se limiter à {math.floor(inverter['max_p_dc']['value']/pv_max_power)} - {math.ceil(inverter['max_p_dc']['value']/pv_max_power)} panneaux tous strings confondus.")
  # md_dis_wr(f"{dc_power_max}")

md_dis_wr("### 3.2.2 Compatibilité en courant")

md_dis_wr(rf"Les panneaux dans les strings étant tous raccordés en série, le courant maximum DC produit sera de ")
md_dis_wr(render_current_panels())

md_dis_wr("### 3.3 Optimisation")

md_dis_wr(rf"L'onduleur fonctionnera de manière optimum avec une tension d'entrée de {inverter['nominal_v_dc']['value']} V.")
md_dis_wr(rf"Dans les conditions NOCT, l'onduleur fournira une tension $V_{{mpp}}$ de {pv_v_mpp_noct} V; le nombre optimum de panneaux par string est dès lors de :")
n_p_optim = round(inverter['nominal_v_dc']['value']/pv_v_mpp_noct,0)
md_dis_wr(rf"$$Optimum = \frac{{{inverter['nominal_v_dc']['value']} (V)}}{{{pv_v_mpp_noct} (V)}} = {n_p_optim}\ Panneaux$$")
if (n_p_optim > n_p_max):
  md_dis_wr(rf"Ce nombre étant supérieur au nombre maximum de panneaux par string calculé précédemment, nous ne retiendrons pas cette valeur **(maximum {n_p_max} panneaux)**.")

md_dis_wr("# 4. Configuration retenue")

md_dis_wr(selected_config_text)

md_dis_wr("# 5. Production estimée")
prod=get_annual_production(panel_repartition,fcs)
annual_prod = sum(prod)
md_dis_wr(f"La production annuelle estimée avec une installation de {panel_number} panneaux ({pv_total_peak_power /1000} Kwc.) est de **{annual_prod} Kwh** calculé comme suit :")

for i in range(len(roof_inclination)):
  md_dis_wr(rf"Toiture {i+1} - Orientation {roof_orientation[i]}:")
  md_dis_wr(rf"$$ Production\ annuelle = {panel_repartition[i]} * {pv_max_power/1000} (kW) * {config_kwh_kwp} * F_c= {panel_repartition[i] * (pv_max_power/1000) * config_kwh_kwp} * {fcs[i]}= {prod[i]} (Kwh)$$")

md_dis_wr("# 6. Détermination des sections de cables")

md_dis_wr(rf"La section des cables est déterminée par la perte maximum de {max_loss*100}%  autorisée pour l'installation : ")
p_max_installation = sum(n_panels_installed) * pv_max_power
max_loss_w = p_max_installation * max_loss
md_dis_wr(rf"$$Perte\ max = {sum(n_panels_installed)} * {pv_max_power} * {max_loss*100}\% = {max_loss_w} W$$")
md_dis_wr(rf"Ces pertes maximum seront constitutées des pertes sur les lignes DC et sur les lignes AC.  La section adéquate de cable sur chaque ligne sera déterminée pour entrer dans la tolérance des {max_loss*100}% ")
md_dis_wr(rf"Enfin spécifiquement pour la ligne AC, la chute de tension maximum de {max_voltage_drop*100}% devra être respectée.")

md_dis_wr("## 6.1 Ligne DC")

md_dis_wr(rf"La section des cables DC sera fonction des pertes de ligne DC calculées comme suit:")
md_dis_wr(rf"$$Pertes_{{DC}} = R*I^2 = \frac{{\rho * l}}{{S}} * I^2$$")
md_dis_wr(rf"Avec : ")
md_dis_wr(f"<ul>")
md_dis_wr(rf"<li><em>R</em> la résistance électrique totale")
md_dis_wr(rf"<li><em>I</em> l'intensité maximum traversant le cable soit le $I_{{mpp}}$ ({pv_i_mpp_stc} A)")
md_dis_wr(rf"<li><em>S</em> la section du cable")
md_dis_wr(rf"<li><em>l</em> sa longueur")
md_dis_wr(rf"<li>$\rho$ la résistivité du cable")
md_dis_wr(rf"Les fabricants de cable DC expriment la résistance électrique de leur matériel par m et par section; ce qui équivaudra au $\rho/S$ : ")
md_dis_wr(rf"<li><em>4 mm²</em> : {round(r_dc_wire_4*1e-3,5)} Ohms/m")
md_dis_wr(rf"<li><em>6 mm²</em> : {round(r_dc_wire_6*1e-3,5)} Ohms/m")
md_dis_wr(f"</ul>")

md_dis_wr(rf"Afin de calculer la perte DC, il faut encore déterminer la longueur de cable DC; ce qui se fait en considérant : ")
md_dis_wr(f"<ul>")
md_dis_wr(rf"<li>La distance onduleur et le string de panneaux")
md_dis_wr(rf"<li>La longueur des brins de cables attachés à chaque panneau")
md_dis_wr(rf"<li>La distance entre le premier et dernier panneau du string considérés posés en ligne (format portrait)")
md_dis_wr(f"</ul>")

table_wire_length_4, l_tot_s, l_tot_wire_panels = compute_and_render_string_length(dc_line_length, n_panels_installed, 4)

md_dis_wr(table_wire_length_4)

l_tot_4 = round(l_tot_s+l_tot_wire_panels,1)
dc_loss_4 = round(r_dc_wire_4*1e-3*l_tot_4*pv_i_mpp_stc*pv_i_mpp_stc,1)
md_dis_wr(rf"La longueur totale en 4mm² de cable est donc de : {l_tot_4}m.  La perte totale des cables DC de section 4mm² est alors de : ")
md_dis_wr(rf"$$Pertes_{{DC}} = {round(r_dc_wire_4*1e-3,5)} Ohms/m * {l_tot_4} m * {pv_i_mpp_stc} A^2 = {dc_loss_4} W$$")

table_wire_length_6, l_tot_s, l_tot_wire_panels = compute_and_render_string_length(dc_line_length, n_panels_installed, 6)
l_tot_6_6= round(l_tot_s,1)
l_tot_6_4= round(l_tot_wire_panels,1)

if (dc_loss_4 > max_loss_w) :
  md_dis_wr(rf"Les pertes DC surpassant les pertes maximum autorisées pour le système, il faut envisager des cables DC de section supérieure (6mm²).  Les longueurs de cables DC sont alors les suivantes : ")
  dc_loss=render_dc_wire_comptutation(table_wire_length_6,l_tot_6_6,l_tot_6_4,pv_i_mpp_stc)
else :
  text_ac_4, ac_4_wire_section = get_ac_max_section(max_loss_w, dc_loss_4, inverter['max_i_ac']['value'], ac_line_length, inverter['ac_output_type']['value'])
  if (ac_4_wire_section > 16) :
    md_dis_wr(rf"Les pertes DC sont trop grandes pour avoir des pertes AC permettant aux pertes systèmes de rester dans les tolérances. Il faut envisager des cables DC de section supérieure (6mm²).  Les longueurs de cables DC sont alors les suivantes : ")
    dc_loss=render_dc_wire_comptutation(table_wire_length_6,l_tot_6_6,l_tot_6_4,pv_i_mpp_stc)
  else :
    dc_loss = dc_loss_4
    md_dis_wr(rf"**La section de cable retenue pour les cables DC est : 4mm²**")

md_dis_wr("## 6.2 Ligne AC")
md_dis_wr("### 6.2.1 Pertes de ligne")
md_dis_wr(rf"Les pertes AC maximum sont déterminées par la différence entre les pertes totales système et les pertes DC")
text_ac, wire_section_ac_loss = get_ac_max_section(max_loss_w, dc_loss, inverter['max_i_ac']['value'], ac_line_length, inverter['ac_output_type']['value'])
md_dis_wr(text_ac)

md_dis_wr("### 6.2.2 Chute de tension ")

text_v_drop, wire_section_v_drop= get_ac_section_for_voltage_drop(inverter['max_i_ac']['value'],ac_line_length, inverter['ac_output_type']['value'])
md_dis_wr(text_v_drop)

md_dis_wr("### 6.2.3 Sélection de la section de ligne AC")

ac_wire_section = max(wire_section_ac_loss, wire_section_v_drop )
md_dis_wr(rf"Considérant les pertes de ligne AC et la chute de tension calculées, la section de cable AC retenue est de **{ac_wire_section} mm²**")

md_dis_wr("# 7. Rendement")
md_dis_wr("## 7.1 Rendement des cables")
ac_loss = get_ac_loss(inverter['max_i_ac']['value'],ac_line_length, ac_wire_section, inverter['ac_output_type']['value'])
total_loss = ac_loss + dc_loss
md_dis_wr(rf"Sur base des sections de cables retenues, on peut recalculer les pertes AC et DC :")
md_dis_wr(f"<ul>")
md_dis_wr(rf"<li>Pertes DC : {dc_loss} W")
md_dis_wr(rf"<li>Pertes AC : {ac_loss} W")
md_dis_wr(f"</ul>")
md_dis_wr(rf"Les pertes totales s'élèvent alors à **{total_loss} W** ce qui est bien inférieur à la perte maximale autorisée de {max_loss_w} W ")
md_dis_wr(rf"Le rendement des cables peut se déterminer sur base de ces pertes et la puissance maximale de l'installation de panneaux ({p_max_installation} W) :")
wire_efficiency = 1-round(total_loss/p_max_installation,4)
md_dis_wr(rf"$$\eta_{{cables}} = 1 - \frac{{Pertes_{{cables}}}}{{P_{{installation}}}} = {wire_efficiency*100} \%$$")

md_dis_wr("# 7.2 Rendement total")
md_dis_wr(rf"Le rendement total de l'installation s'obtient en combinant le rendement de chacun des composants : ")
md_dis_wr(f"<ul>")
md_dis_wr(rf"<li>Rendement des modules $\eta_{{modules}}$: {pv_module_efficiency*100}% (datasheet panneaux)")
md_dis_wr(rf"<li>Rendement onduleur $\eta_{{onduleur}}$: {inverter['efficiency']['value']*100}% (datasheet onduleur)")
md_dis_wr(rf"<li>Rendement des cables $\eta_{{cables}}$: {wire_efficiency*100}% (calculé)")
md_dis_wr(rf"<li>Rendement fiches MC4 $\eta_{{fiches}}$: {plug_efficiency*100}% (estimé)")
md_dis_wr(rf"<li>Rendement compteur vert $\eta_{{compteur}}$: {meter_efficiency*100}% (estimé)")
md_dis_wr(rf"<li>Rendement installation électrique $\eta_{{installation\ elec}}$: {electric_install_efficiency*100}% (estimé)")
md_dis_wr(f"</ul>")
md_dis_wr(rf"De la manière suivante : ")
total_efficiency = round(pv_module_efficiency*inverter['efficiency']['value']*wire_efficiency*plug_efficiency*meter_efficiency*electric_install_efficiency,4)
md_dis_wr(rf"$$\eta_{{total}} = \eta_{{modules}} * \eta_{{onduleur}} * \eta_{{cables}} * \eta_{{fiches}} * \eta_{{compteur}} * \eta_{{installation\ elec}} = {round(total_efficiency*100,2)} \%$$")
md_dis_wr(rf"En d'autres termes, considérant une irradiance de 1000W/m², les panneaux récupéreront {round(1000*pv_module_efficiency,2)} W et le système permettra de réinjecter la puissance de {1000*total_efficiency}W.")
md_dis_wr(rf"Cela représente donc une perte de {round(((pv_module_efficiency/total_efficiency)-1),2)*100} \%.")

close_md_file()

Checking for the existence of file: 'pv_report.md'
File 'pv_report.md' existed and was successfully deleted.
Checking for the existence of file: 'pv_report_output.md'
File 'pv_report_output.md' existed and was successfully deleted.


# 1. Description de l'installation

Le client projette une consommation de 5000 Kwh.

Les surfaces de toiture exploitables sont les suivantes :     

| Surface | Inclinaison (°)| Orientation | Repartition (%) |
|---------|---------------|-------------|---------------|
| 1       | 35          | Ouest   | 50.0        |
| 2       | 35          | Est   | 50.0        |


...

</ul>

De la manière suivante : 

$$\eta_{total} = \eta_{modules} * \eta_{onduleur} * \eta_{cables} * \eta_{fiches} * \eta_{compteur} * \eta_{installation\ elec} = 20.64 \%$$

En d'autres termes, considérant une irradiance de 1000W/m², les panneaux récupéreront 227.9 W et le système permettra de réinjecter la puissance de 206.4W.

Cela représente donc une perte de 10.0 \%.